In [63]:
import requests
import pandas as pd
import time
import datetime
from prefect.blocks.system import Secret
import os
from prefect.blocks.system import Secret
from prefect.filesystems import GCS
from prefect_gcp.cloud_storage import GcsBucket

In [36]:
headers = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7",
    "Accept-Charset": "application/x-www-form-urlencoded; charset=UTF-8",
    "Origin": "https://developer.riotgames.com",
    "X-Riot-Token": ""
}


In [64]:

def write_gcs(path: str):
    gcp_block = GcsBucket.load("lol-datalake")
    gcp_block.upload_from_path(
        from_path=f'{path}',
        to_path='data/'
    )


In [46]:
df = pd.read_csv('/home/guilhermefmk/workspace/DEzoomcamp_Final_project/data_flow/chalenger_14-04-2023.csv')

In [37]:
response = requests.get('https://br1.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/CHALLENGER/I?page=1', headers=headers)

In [42]:

def write_local(df: pd.DataFrame) -> str:
    '''Write DataFrame out locally as parquet file'''
    dia = datetime.datetime.now().day
    mes = datetime.datetime.now().month
    ano = datetime.datetime.now().year
    dataset_file = f'chalenger_{dia:02}-{mes:02}-{ano:02}'
    folder_path = './data_flow'
    dir_path = os.path.join(folder_path)
    path = os.path.join(dir_path, dataset_file + '.csv')


    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

    df.to_csv(path)
    return path

In [39]:
df = pd.read_json(response.text,orient='records')

In [43]:
write_local(df)

'./data_flow/chalenger_14-04-2023.csv'

In [50]:
df.dtypes

leagueId        object
queueType       object
tier            object
rank            object
summonerId      object
summonerName    object
leaguePoints     int64
wins             int64
losses           int64
veteran           bool
inactive          bool
freshBlood        bool
hotStreak         bool
puuid           object
dtype: object

In [25]:
def get_puuid(summoner_id):
    url = f'https://br1.api.riotgames.com/lol/summoner/v4/summoners/{summoner_id}'
    response = requests.get(url, headers=headers)
    response_json = response.json()
    print(response_json)
    return response_json['puuid']

In [31]:
puuids = []
for index, row in df.iterrows():
    time.sleep(3)
    puuid = get_puuid(row['summonerId'])
    puuids.append(puuid)

df = df.assign(puuid=puuids)

{'id': 'oMe8xpXahTZEVIElT6SN80BTa2UPkvWs3-gbleXZ8wx7Ox0', 'accountId': 'QWJ1blQAAoqfwGh7FRGVQK4w-WISHbr-ROMQxAv7xKcWVxQ', 'puuid': 'dobcFjMdScI2VsSaqr8SIIc-4gLafx-qmdEwP45Qm-hOPkVIhEP7Wwvq9rNFxZ9NZyNSoA1Xo0qoAQ', 'name': 'frosty', 'profileIconId': 5490, 'revisionDate': 1681408179030, 'summonerLevel': 813}
{'id': 'iWs_XHNMQHNf0ofFqWOiirYcxh1r90yVgZZ1IDw_dH8sj-i_pkLQEg4I0g', 'accountId': '6YIWr_SQ2ATmURA97oqe73bG2RNoy_Zo4HOqXPYe8zv7GdRs6IApaMTK', 'puuid': 'IzjFQWI5jxL8gV_hY6mJ2KGPIpYGTmwtu67B-FPNsLFw5R0bPf5iwohdgRioveL3F__1bOTgALy00A', 'name': 'FURIA Ayu', 'profileIconId': 29, 'revisionDate': 1681334111999, 'summonerLevel': 93}
{'id': 'XfCU7vQVC3EE8LN-vOiVqVJUBqYYyzVcg7sfj9L8UAGWz3k', 'accountId': 'lqI2cfkApSY2RSxsQx2if2R5P5hLyztmMhi86N9gqryzjgc', 'puuid': 'Q6H4ALKdhPtAuY5mbYnb8sGJH3cs5SL3_ayZNnqVS5OT_MRteqo2P3bCGv8KX6gBh7c08sm1_r2dCQ', 'name': 'Tutsz', 'profileIconId': 3008, 'revisionDate': 1680491665000, 'summonerLevel': 717}
{'id': '1IHjZOFkFSBMwBzKnrzfIUKK74RuoPg4SiRIfNurHId4Re0', 'a

In [45]:
def get_match_ids(puuid: str) -> list:
    url = f'https://americas.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start=0&count=100'
    response = requests.get(url, headers=headers)
    match_ids_list = []
    for id in response.json():
        match_ids_list.append(id)
    return match_ids_list


In [46]:
matches_ids_global_raw = []
for index, row in df.iterrows():
    time.sleep(3)
    matches_ids_especific_summoner = get_match_ids(row['puuid'])
    matches_ids_global_raw.extend(matches_ids_especific_summoner)


In [49]:
print(len(set(matches_ids_global_raw)))

14257


In [ ]:
def get_match_info(match_id: str):
    url = f'https://americas.api.riotgames.com/lol/match/v5/matches/{match_id}'
    response = requests.get(url, headers=headers)
    for data in response.json():
                participant_data = {
                'gameId': data['info']['gameId'],
                'gameMode': data['info']['gameMode'],
                'gameType': data['info']['gameType'],
                'id': participant['id'],
                'name': participant['name'],
                'score': participant['score']
            }
    return match_ids_list

In [52]:
import json

with open('response.json') as f:
    dados = json.load(f)

In [60]:
url = f'https://americas.api.riotgames.com/lol/match/v5/matches/BR1_2712884156'
response = requests.get(url, headers=headers)
dados = response.json()
print(dados['info'])


{'gameCreation': 1681406272102, 'gameDuration': 1805, 'gameEndTimestamp': 1681408169958, 'gameId': 2712884156, 'gameMode': 'CLASSIC', 'gameName': 'teambuilder-match-2712884156', 'gameStartTimestamp': 1681406364873, 'gameType': 'MATCHED_GAME', 'gameVersion': '13.7.503.1019', 'mapId': 11, 'participants': [{'allInPings': 0, 'assistMePings': 0, 'assists': 1, 'baitPings': 0, 'baronKills': 0, 'basicPings': 0, 'bountyLevel': 0, 'challenges': {'12AssistStreakCount': 0, 'abilityUses': 377, 'acesBefore15Minutes': 0, 'alliedJungleMonsterKills': 9, 'baronTakedowns': 1, 'blastConeOppositeOpponentCount': 0, 'bountyGold': 0, 'buffsStolen': 1, 'completeSupportQuestInTime': 0, 'controlWardsPlaced': 0, 'damagePerMinute': 445.1003236896544, 'damageTakenOnTeamPercentage': 0.17550502806974208, 'dancedWithRiftHerald': 0, 'deathsByEnemyChamps': 8, 'dodgeSkillShotsSmallWindow': 1, 'doubleAces': 0, 'dragonTakedowns': 0, 'earliestBaron': 1247.3092287860177, 'earlyLaningPhaseGoldExpAdvantage': 0, 'effectiveHealA

In [61]:
game_id = dados['info']['gameId']
game_mode = dados['info']['gameMode']
game_type = dados['info']['gameType']
participantes = dados['info']['participants']

lista_participantes = []
for p in participantes:
    d = {'gameId': game_id, 'gameMode': game_mode, 'gameType': game_type}
    d.update(p)
    lista_participantes.append(d)


In [62]:
df = pd.DataFrame(lista_participantes)

In [63]:
df.head()

,gameId,gameMode,gameType,allInPings,assistMePings,assists,baitPings,baronKills,basicPings,bountyLevel,...,turretKills,turretTakedowns,turretsLost,unrealKills,visionClearedPings,visionScore,visionWardsBoughtInGame,wardsKilled,wardsPlaced,win
0,2712884156,CLASSIC,MATCHED_GAME,0,0,1,0,0,0,0,...,3,3,7,0,0,19,0,3,8,False
1,2712884156,CLASSIC,MATCHED_GAME,0,9,7,1,1,0,0,...,0,0,7,0,0,17,0,3,0,False
2,2712884156,CLASSIC,MATCHED_GAME,0,1,9,0,0,0,0,...,0,0,7,0,0,33,6,4,16,False
3,2712884156,CLASSIC,MATCHED_GAME,0,20,8,0,0,0,0,...,0,1,7,0,0,32,3,9,12,False
4,2712884156,CLASSIC,MATCHED_GAME,0,6,20,0,0,0,0,...,0,1,7,0,0,86,14,9,42,False
